In [28]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
import numpy as np

In [29]:
# Отключение экспоненциальной записи для всех DataFrame
pd.set_option('display.float_format', '{:.6f}'.format)
# Отключение экспоненциальной записи в numpy
np.set_printoptions(suppress=True)

In [2]:
anime, rating = pd.read_csv('anime\\anime.csv'), pd.read_csv('anime\\rating.csv')
anime.shape, rating.shape

((12294, 7), (7813737, 3))

Info

In [5]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [6]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [3]:
rating.dropna(inplace=True)
anime.dropna(inplace=True)

AGG

In [4]:
mean_rating = rating.groupby('user_id')['rating'].transform('mean')
mean_rating

0         -0.712418
1         -0.712418
2         -0.712418
3         -0.712418
4         -0.712418
             ...   
7813732    7.719388
7813733    7.719388
7813734    7.719388
7813735    9.000000
7813736    9.000000
Name: rating, Length: 7813737, dtype: float64

In [5]:
new_rating = rating[rating['rating'] >= mean_rating]
new_rating

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
...,...,...,...
7813730,73515,13659,8
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [6]:
scaler = LabelEncoder()
data = scaler.fit_transform(anime['type'])

In [7]:
new_anime = pd.concat([anime.reset_index(), pd.DataFrame({"Type": data})], axis=1).drop(['type', 'index'], axis=1)
new_anime

,anime_id,name,genre,episodes,rating,members,Type
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",1,9.37,200630,0
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",64,9.26,793665,5
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",51,9.25,114262,5
3,9253,Steins;Gate,"Sci-Fi, Thriller",24,9.17,673572,5
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",51,9.16,151266,5
...,...,...,...,...,...,...,...
12012,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,1,4.15,211,3
12013,5543,Under World,Hentai,1,4.28,183,3
12014,5621,Violence Gekiga David no Hoshi,Hentai,4,4.88,219,3
12015,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,1,4.98,175,3


In [8]:
rating_merge = pd.merge(new_rating, new_anime, on='anime_id', how='inner').drop(['anime_id', 'name'], axis=1).set_index('user_id')
rating_merge

,rating_x,genre,episodes,rating_y,members,Type
user_id,,,,,,
1,10,"Action, Ecchi, Horror, Supernatural",12,7.46,535892,5
1,10,"Comedy, Demons, Ecchi, Harem, Romance, School",12,7.70,398660,5
1,10,"Action, Adventure, Fantasy, Game, Romance",25,7.83,893100,5
1,10,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",12,7.87,266657,5
2,10,"Comedy, School, Shounen, Sports",25,8.46,338315,5
...,...,...,...,...,...,...
73515,8,"Comedy, Seinen, Slice of Life",13,7.43,194791,5
73515,9,"Mecha, Police, Psychological, Sci-Fi",1,7.64,31747,0
73515,10,"Comedy, Demons, Fantasy, Historical, Shounen, ...",10,8.37,122895,5


In [9]:
rating_merge_sample = rating_merge.sample(frac=0.5, random_state=42)
rating_merge_sample

,rating_x,genre,episodes,rating_y,members,Type
user_id,,,,,,
64439,8,"Comedy, Ecchi, Harem, Romance, Shounen, Slice ...",24,7.28,156627,5
45297,9,"Adventure, Comedy, Fantasy, Supernatural",10,8.03,244877,5
11110,10,"Mystery, Romance, Supernatural, Vampire",15,8.39,482268,5
54524,10,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26,8.82,486824,5
2444,8,"Comedy, Drama, Romance, School",13,8.12,353876,5
...,...,...,...,...,...,...
3583,8,"Action, Magic, Sci-Fi, Super Power",1,7.71,54873,0
5826,10,"Comedy, Drama, School, Shounen, Slice of Life",43,8.77,268487,5
45659,7,"Comedy, Ecchi, School, Seinen",1,7.18,5553,3


In [11]:
# Использование get_dummies()
genre_dummies = rating_merge_sample['genre'].str.get_dummies(sep=', ')
genre_dummies

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
user_id,,,,,,,,,,,,,,,,,,,,,
64439,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
45297,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
11110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
54524,1,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2444,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5826,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
45659,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Объединение с исходным DataFrame
df_with_dummies = pd.concat([rating_merge_sample.drop('genre', axis=1), genre_dummies], axis=1)
df_with_dummies

,rating_x,episodes,rating_y,members,Type,Action,Adventure,Cars,Comedy,Dementia,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
user_id,,,,,,,,,,,,,,,,,,,,,
64439,8,24,7.28,156627,5,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
45297,9,10,8.03,244877,5,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
11110,10,15,8.39,482268,5,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
54524,10,26,8.82,486824,5,1,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2444,8,13,8.12,353876,5,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,8,1,7.71,54873,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5826,10,43,8.77,268487,5,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
45659,7,1,7.18,5553,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Преобразование всех числовых колонок в меньшие типы данных
for col in df_with_dummies.columns:
    if df_with_dummies[col].dtype == 'float64':
        df_with_dummies[col] = df_with_dummies[col].astype('float32')
    elif df_with_dummies[col].dtype == 'int64':
        df_with_dummies[col] = df_with_dummies[col].astype('int32')

df_with_dummies.dtypes

rating_x           int32
episodes          object
rating_y         float32
members            int32
Type               int32
Action             int32
Adventure          int32
Cars               int32
Comedy             int32
Dementia           int32
Demons             int32
Drama              int32
Ecchi              int32
Fantasy            int32
Game               int32
Harem              int32
Hentai             int32
Historical         int32
Horror             int32
Josei              int32
Kids               int32
Magic              int32
Martial Arts       int32
Mecha              int32
Military           int32
Music              int32
Mystery            int32
Parody             int32
Police             int32
Psychological      int32
Romance            int32
Samurai            int32
School             int32
Sci-Fi             int32
Seinen             int32
Shoujo             int32
Shoujo Ai          int32
Shounen            int32
Shounen Ai         int32
Slice of Life      int32


In [17]:
df_with_dummies['episodes'] = pd.to_numeric(df_with_dummies['episodes'], errors='coerce')

In [46]:
# Разделение данных на признаки (X) и целевую переменную (y)
X = df_with_dummies.drop(['rating_x'], axis=1)
y = df_with_dummies['rating_x']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# Преобразование данных в формат LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Параметры модели
params = {
    'objective': 'regression',  # Так как мы предсказываем рейтинг
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1
}

# Использование обратного вызова для ранней остановки
callbacks = [lgb.early_stopping(stopping_rounds=10)]

# Тренировка модели
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=1000, callbacks=callbacks)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Оценка качества модели
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[150]	valid_0's rmse: 2.90056
RMSE: 2.9005640219724187


d:\anaconda\envs\Python310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


PCA

In [44]:
# data = PCA(n_components=4).fit_transform(X)
# X_pca = pd.DataFrame(data, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4'])
# # Разделение на тренировочную и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [45]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = model1.predict(X_test)

# Оценка качества модели
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

RMSE: 2.9417415197797476


d:\anaconda\envs\Python310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
